In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow
from tensorflow.keras.layers import Conv2D , MaxPooling2D , Dense , Input , Dropout , Flatten , BatchNormalization
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.regularizers import L2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [ ]:
train_data=pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
test_data=pd.read_csv('/kaggle/input/digit-recognizer/test.csv')
test=pd.read_csv('/kaggle/input/digit-recognizer/test.csv')

In [ ]:
X = train_data.drop("label", axis=1).values / 255.0 
y = to_categorical(train_data["label"].values, num_classes=10) 
test_data= test_data.values / 255.0

In [ ]:
X.shape

In [ ]:
n_samples = (42000*784) // (28*28) 
n_samples #=42000

In [ ]:
X=X.reshape(-1,28,28,1)
test_data=test_data.reshape(-1,28,28,1)

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)

In [ ]:
X_train.shape[0] // 128

In [ ]:
datagen = tensorflow.keras.preprocessing.image.ImageDataGenerator(
                            featurewise_center=False,
                            samplewise_center=False,
                            featurewise_std_normalization=False,
                            samplewise_std_normalization=False,
                            zca_whitening=False,
                            zca_epsilon=1e-06,
                            rotation_range=20,
                            width_shift_range=0.2,
                            height_shift_range=0.2,
                            brightness_range=None,
                            shear_range=0.0,
                            zoom_range=0.2,
                            channel_shift_range=0.0,
                            fill_mode='nearest',
                            cval=0.0,
                            horizontal_flip=False,
                            vertical_flip=False,
                            rescale=None,
                            preprocessing_function=None,
                            data_format=None,
                            interpolation_order=1,
                            dtype=None
)

datagen.fit(X_train)


model=Sequential([
    Input(shape=(28,28,1)),
    Conv2D(32,(3,3),activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    Conv2D(64,(3,3),activation='relu'),
    Conv2D(64,(2,2),activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    Flatten(),

    Dense(64 ,activation='relu'),
    Dense(64 ,activation='relu'),
    
    Dense(10 ,activation='softmax')
])
model.compile(optimizer=Adam(learning_rate=0.001), metrics=['accuracy'],loss='categorical_crossentropy')

lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)

history=model.fit(datagen.flow(X_train,y_train,128),
                  epochs=60,callbacks=EarlyStopping(),
                  steps_per_epoch=300,validation_data=(X_test, y_test),
                  validation_split=0.2)


In [ ]:

def test_result(model, n, x):
    tta_steps = n  
    predictions = np.zeros((x.shape[0], 10))  # For storing cumulative predictions

    tta_datagen = tensorflow.keras.preprocessing.image.ImageDataGenerator(
        rotation_range=20, 
        zoom_range=0.2,  
        width_shift_range=0.2, 
        height_shift_range=0.2,
        validation_split=0.2
    )
    
    print("\nTest-Time Augmentation")
    for i in range(tta_steps):
        print(f"TTA round {i+1}/{tta_steps}")  
        test_augmented = tta_datagen.flow(x, shuffle=False, batch_size=64)  
        predictions += model.predict(test_augmented, verbose=0)  

    predictions /= tta_steps 
    y_final = np.argmax(predictions, axis=1)
    
    return y_final
from sklearn.metrics import accuracy_score


y_final = test_result(model, 10, X_test)

accuracy = accuracy_score(np.argmax(y_test, axis=1), y_final)
print(f"Accuracy after TTA: {accuracy * 100:.2f}%")


In [ ]:
y_final

In [ ]:

plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.show()

plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt


def test_prediction(imageNum, model):
    
    image = X_test[imageNum]
    true_label = y_test[imageNum]  


    current_image = image.reshape((28, 28)) * 255  

   
    prediction = model.predict(image.reshape(1, 28, 28, 1)) 

   
    plt.gray()
    plt.imshow(current_image, interpolation='nearest')
    plt.show()

    predicted_label = np.argmax(prediction, axis=1) 
    true_label_index = np.argmax(true_label)  

    print(f"Prediction: {predicted_label[0]}")
    print(f"Label: {true_label_index}")


for i in range(5):
    imageNum = np.random.randint(0, len(X_test))  
    test_prediction(imageNum, model)  


In [ ]:
y_prediction = test_result(model, 10, test_data)

In [ ]:
y_prediction

In [ ]:
test

In [ ]:

y_pred = pd.DataFrame(y_prediction, columns=['label'])
y_pred

In [ ]:
ID = pd.read_csv('/kaggle/input/digit-recognizer/sample_submission.csv')

image_id_df = ID[['ImageId']]

In [ ]:
image_id_df 

In [ ]:
submission= pd.concat([image_id_df,y_pred],axis=1)
submission

In [ ]:
submission.to_csv("MNISTsubmission.csv",index=False)